In [1]:
import torch
import numpy as np
from datagen import *
from datasets import *
from modules import *
from backdoor import Backdoor

/home/andmholm/Backdoor/backdoor_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
backdoor = Backdoor('convnet', '11-layer', channels=3, classes=10)

In [3]:
backdoor.load_data('cfar10', 'image')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
backdoor.create_models('sgd', 'crossentropy', lr=0.01)

In [5]:
patches = ImagePatch((9, 9), 3, 'random').get_patches(len(backdoor.traindata))

In [6]:
targets = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 0}
operations = (None, [RotateTransform()], [LabelTransform(targets)])

In [7]:
backdoor.poison(patches, operations, OverlayMerge(select=True), 0.2)

/home/andmholm/Backdoor/experiments/datagen.py:458: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(modified)


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
backdoor.train('both', epochs=1, verbose=True, device=device)

Net ready for training
Training started
Epoch 1/1 | 24.95% | Loss: 2.0926 | Samples trained: 12480/50000
Epoch 1/1 | 49.90% | Loss: 1.9622 | Samples trained: 24960/50000
Epoch 1/1 | 74.86% | Loss: 1.8870 | Samples trained: 37440/50000
Epoch 1/1 | 99.81% | Loss: 1.8237 | Samples trained: 49920/50000
Epoch 1 complete | Loss: 1.8234
Training complete | Net Average Loss: 1.8234 | Total epochs: 1
Net ready for training
Training started
Epoch 1/1 | 24.95% | Loss: 2.1763 | Samples trained: 12480/50000
Epoch 1/1 | 49.90% | Loss: 2.0758 | Samples trained: 24960/50000
Epoch 1/1 | 74.86% | Loss: 2.0149 | Samples trained: 37440/50000
Epoch 1/1 | 99.81% | Loss: 1.9624 | Samples trained: 49920/50000
Epoch 1 complete | Loss: 1.9622
Training complete | Net Average Loss: 1.9622 | Total epochs: 1


(1.8234093831429021, 1.9621771014964664)

In [10]:
backdoor.eval(device=device)

Net ready for evaluation
Testing started
24.92% Testing complete | Loss: 1.6268 | Accuracy: 0.3970
49.84% Testing complete | Loss: 1.6407 | Accuracy: 0.3882
74.76% Testing complete | Loss: 1.6571 | Accuracy: 0.3789
99.68% Testing complete | Loss: 1.6558 | Accuracy: 0.3803
Testing complete | Loss: 1.6561 | Accuracy: 0.3801
Net ready for evaluation
Testing started
24.92% Testing complete | Loss: 1.9232 | Accuracy: 0.2416
49.84% Testing complete | Loss: 1.9350 | Accuracy: 0.2480
74.76% Testing complete | Loss: 1.9364 | Accuracy: 0.2456
99.68% Testing complete | Loss: 1.9364 | Accuracy: 0.2433
Testing complete | Loss: 1.9370 | Accuracy: 0.2431


(1.6560612345655885, 0.3801, 1.9369513836150734, 0.2431)